# Data Challenge 20221121

Desafio de dados focado em fazer uma análise exploratória de duas bases de dados disponibilizadas em formato csv.

- Lista de filmes da Netflix: netflix_titles.csv

- Lista de filmes da Amazon Prime: amazon_prime_titles.csv

## Filtro de dados

Nessa etapa você poderá exibir os dados por meio de uma REST API ou no Colab, Jupyter Notebook ou similar. Exiba as seguintes informações:

1 - Top 10 atores/atrizes considerando todos os dados;

2 - Top 5 países produtores de conteúdos considerando todos os dados e comparando as duas plataformas;

3 - Mês no qual há mais adições de filmes na plataforma Netflix;

4 - Quantidade de filmes listados como comédia.

5 - Lista de todos os gêneros de filmes.

6 - A frequência de "TV Show" de todos os dados e comparativamente em relação as duas plataformas

7 - A frequência de "Movies" de todos os dados e comparativamente em relação as duas plataformas

In [2]:
import pandas as pd

In [3]:
df_amazon_prime = pd.read_csv(r"data\amazon_prime_titles.csv",sep=",",encoding="utf-8")
df_netflix = pd.read_csv(r"data\netflix_titles.csv",sep=',',encoding="utf-8")

In [4]:
#Transformações
df_amazon_prime['company'] = "Prime"
df_netflix['company'] = "Netflix"

df_total_companys = pd.concat([df_amazon_prime,df_netflix],ignore_index=True)
df_total_companys['country_list'] = df_total_companys['country'].str.split(',')
df_total_companys['cast_list'] = df_total_companys['cast'].str.split(',')

df_total_companys = df_total_companys.dropna(subset=['cast_list','country_list'])

df_total_companys = df_total_companys.explode(['country_list'])
df_total_companys = df_total_companys.explode(['cast_list'])


#Tratamento
df_total_companys['country_list'] = df_total_companys['country_list'].str.strip()
df_total_companys['cast_list'] = df_total_companys['cast_list'].str.strip()

df_total_companys['year_added'] = df_total_companys['date_added'].str.split(",",expand=True)[1]
df_total_companys['month_added'] = df_total_companys['date_added'].str.split(",",expand=True)[0].str.split(" ",expand=True)[0]

In [5]:
#1 - Top 10 atores/atrizes considerando todos os dados

df_compare_country_company = df_total_companys.groupby(by=["cast_list"])['show_id'].count().reset_index().sort_values('show_id',ascending=False).rename(columns={"show_id":"count"})

print("======= TOP 5 ATORES =======\n")
print(df_compare_country_company.head(10),"\n")

======= TOP 5 ATORES =======

                cast_list  count
2837          Anupam Kher     52
30292      Shah Rukh Khan     46
7830   David Attenborough     45
25262             Om Puri     44
24060    Naseeruddin Shah     40
32590         Tara Strong     39
15644         John Cleese     39
34969        Willem Dafoe     38
19578         Liam Neeson     37
834          Akshay Kumar     37 



In [6]:
# 2 - Top 5 países produtores de conteúdos considerando todos os dados e comparando as duas plataformas
df_compare_country_company = df_total_companys.groupby(by=["company","country_list"])['show_id'].count().reset_index().sort_values('show_id',ascending=False).rename(columns={"show_id":"count"})


print("======= TOP 5 PAÍSES PRODUTORES DE CONTÉUDO NETFLIX =======\n")
print(df_compare_country_company[df_compare_country_company['company'] == 'Netflix'].head(5),"\n")
print("======= TOP 5 PAÍSES PRODUTORES DE CONTÉUDO PRIME VIDEO =======\n")
print(df_compare_country_company[df_compare_country_company['company'] == 'Prime'].head(5))

======= TOP 5 PAÍSES PRODUTORES DE CONTÉUDO NETFLIX =======

     company    country_list  count
109  Netflix   United States  27063
44   Netflix           India   7750
108  Netflix  United Kingdom   5332
19   Netflix          Canada   3519
52   Netflix           Japan   3453 

======= TOP 5 PAÍSES PRODUTORES DE CONTÉUDO PRIME VIDEO =======

    company    country_list  count
159   Prime   United States   2110
135   Prime           India   1724
158   Prime  United Kingdom    363
132   Prime         Germany    225
123   Prime          Canada    185


In [7]:
# 3 - Mês no qual há mais adições de filmes na plataforma Netflix

mes_maior_adicao = df_total_companys[df_total_companys["company"] == 'Netflix']\
    .groupby("month_added")["show_id"].count()\
    .sort_values(ascending=False).head(1).index[0]

print(df_total_companys[df_total_companys["company"] == 'Netflix'].groupby("month_added")["show_id"].count().sort_values(ascending=False))

print(f"-------------------------\nMês com maior adição de filmes: {mes_maior_adicao}")


month_added
January      7423
December     6915
July         6911
September    6679
October      6672
November     6342
April        6258
August       6202
March        6022
June         5870
May          4907
February     4826
              741
Name: show_id, dtype: int64
-------------------------
Mês com maior adição de filmes: January


In [8]:
#4 - Quantidade de filmes listados como comédia.
total_comedy = len(df_total_companys[df_total_companys["listed_in"].str.contains("Comedy")])

print(f"Total de filmes listados como Comedy: {total_comedy}")


Total de filmes listados como Comedy: 2007


In [9]:
#5 - Lista de todos os gêneros de filmes.
for i in df_total_companys["listed_in"].str.split(", ").explode().unique().tolist():
    print(i)

Comedy
Drama
International
Action
Suspense
Documentary
Fantasy
Kids
Special Interest
Science Fiction
Adventure
Horror
Sports
TV Shows
Unscripted
Arts
Entertainment
and Culture
Romance
Animation
LGBTQ
Young Adult Audience
Military and War
Music Videos and Concerts
Historical
Arthouse
Western
Talk Show and Variety
Faith and Spirituality
Anime
International TV Shows
TV Dramas
TV Mysteries
Romantic TV Shows
TV Comedies
Dramas
Independent Movies
International Movies
British TV Shows
Reality TV
Comedies
Crime TV Shows
Spanish-Language TV Shows
TV Action & Adventure
Romantic Movies
Docuseries
Horror Movies
Sci-Fi & Fantasy
Thrillers
Kids' TV
Action & Adventure
TV Sci-Fi & Fantasy
Classic Movies
Anime Features
Anime Series
Children & Family Movies
Documentaries
Music & Musicals
Korean TV Shows
Teen TV Shows
Cult Movies
Faith & Spirituality
LGBTQ Movies
TV Horror
Stand-Up Comedy
Movies
Sports Movies
Classic & Cult TV
TV Thrillers
Science & Nature TV
Stand-Up Comedy & Talk Shows


In [24]:
#6-A frequência de "TV Show" de todos os dados e comparativamente em relação as duas plataformas

total_ctv_shows = len(df_total_companys[df_total_companys["type"] == "TV Show"])

print(f"\n Total de Tv Shows: {total_ctv_shows}\n")
print("Total tv shows por Empresa\n")
print(df_total_companys[df_total_companys["type"] == "TV Show"].groupby("company")['show_id'].count())


 Total de Tv Shows: 21415

Total tv shows por Empresa

company
Netflix    20550
Prime        865
Name: show_id, dtype: int64


In [25]:
#7 - A frequência de "Movies" de todos os dados e comparativamente em relação as duas plataformas

total_ctv_shows = len(df_total_companys[df_total_companys["type"] == "Movie"])

print(f"\n Total de Tv Shows: {total_ctv_shows}\n")
print("Total tv shows por Empresa\n")
print(df_total_companys[df_total_companys["type"] == "Movie"].groupby("company")['show_id'].count())


 Total de Tv Shows: 59675

Total tv shows por Empresa

company
Netflix    55286
Prime       4389
Name: show_id, dtype: int64
